### **Mise En Garde: Certaines méthodes sont exploitables en fonction de la version de Python que vous ustilisez**

In [ ]:
!pip install pyspark

In [1]:
import pyspark
print(f"{pyspark.__version__}")

3.5.0


In [3]:
import os
os.listdir(os.getcwd())

['.config',
 'DataFrames_sample.csv',
 'airport-codes-na.txt',
 'DataFrames_sample.json',
 'departuredelays.csv',
 'tmp',
 'sample_data']

### **Point d'entrée : SparkContext**

In [4]:
sc = pyspark.SparkContext()

# Verifier si SparkContext est valide
print(sc)

<SparkContext master=local[*] appName=pyspark-shell>


In [5]:
# Check the version of SparkContext in PySpark Shell
print("La version de Spark Context dans PySpark Shell est : ", sc.version)

# Display the Python version of SparkContext
print("La version Python de Spark Context dans PySpark Shell est : ", sc.pythonVer)

# Display the master of SparkContext
print("Le maitre de Spark Context dans PySpark Shell est : ", sc.master)

La version de Spark Context dans PySpark Shell est :  3.5.0
La version Python de Spark Context dans PySpark Shell est :  3.10
Le maitre de Spark Context dans PySpark Shell est :  local[*]


### **Create Some Data**

In [6]:
sample_data = sc.parallelize([
        (1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD', 13.75, 9.48, 0.61, 4.02),
        (2, 'MacBook', 2016, '12"', '8GB', '256GB SSD', 11.04, 7.74, 0.52, 2.03),
        (3, 'MacBook Air', 2016, '13.3"', '8GB', '128GB SSD', 12.8, 8.94, 0.68, 2.96),
        (4, 'iMac', 2017, '27"', '64GB', '1TB SSD', 25.6, 8.0, 20.3, 20.8)
        ])

### **Creating DataFrames**

**SparkSession : Point d'entrée pour les Dataframes**

In [7]:
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [8]:
sample_data_df = spark.createDataFrame(
    sample_data,
    [
        'Id',
        'Model',
        'Year',
        'ScreenSize',
        'RAM',
        'HDD',
        'W',
        'D',
        'H',
        'Weight'
    ]
)

In [9]:
sample_data.take(1)

[(1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD', 13.75, 9.48, 0.61, 4.02)]

In [10]:
sample_data_df.take(1)

[Row(Id=1, Model='MacBook Pro', Year=2015, ScreenSize='15"', RAM='16GB', HDD='512GB SSD', W=13.75, D=9.48, H=0.61, Weight=4.02)]

In [11]:
sample_data_df.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [12]:
sample_data_df.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- HDD: string (nullable = true)
 |-- W: double (nullable = true)
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- Weight: double (nullable = true)



### **From JSON**


In [13]:
sample_data_json_df = (
    spark
    .read
    .json('DataFrames_sample.json')
)

In [14]:
sample_data_json_df.show()

+----+----+---------+-----------+----+-----------+-----+-------+-----+---+
|   D|   H|      HDD|      Model| RAM| ScreenSize|    W| Weight| Year| Id|
+----+----+---------+-----------+----+-----------+-----+-------+-----+---+
|9.48|0.61|512GB SSD|MacBook Pro|16GB|        15"|13.75|   4.02| 2015|  1|
|7.74|0.52|256GB SSD|    MacBook| 8GB|        12"|11.04|   2.03| 2016|  2|
|8.94|0.68|128GB SSD|MacBook Air| 8GB|      13.3"| 12.8|   2.96| 2016|  3|
| 8.0|20.3|  1TB SSD|       iMac|64GB|        27"| 25.6|   20.8| 2017|  4|
+----+----+---------+-----------+----+-----------+-----+-------+-----+---+



In [15]:
sample_data_json_df.printSchema()

root
 |--  D: double (nullable = true)
 |--  H: double (nullable = true)
 |--  HDD: string (nullable = true)
 |--  Model: string (nullable = true)
 |--  RAM: string (nullable = true)
 |--  ScreenSize: string (nullable = true)
 |--  W: double (nullable = true)
 |--  Weight: double (nullable = true)
 |--  Year: long (nullable = true)
 |-- Id: long (nullable = true)



### **FROM CSV**

In [16]:
sample_data_csv = (
    spark
    .read
    .csv('DataFrames_sample.csv',
         header = True,
        inferSchema = True)

)

In [17]:
sample_data_csv.show()

+---+-----------+-----+-----------+----+---------+-----+----+----+-------+
| Id|      Model| Year| ScreenSize| RAM|      HDD|    W|   D|   H| Weight|
+---+-----------+-----+-----------+----+---------+-----+----+----+-------+
|  1|MacBook Pro| 2015|        15"|16GB|512GB SSD|13.75|9.48|0.61|   4.02|
|  2|    MacBook| 2016|        12"| 8GB|256GB SSD|11.04|7.74|0.52|   2.03|
|  3|MacBook Air| 2016|      13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|   2.96|
|  4|       iMac| 2017|        27"|64GB|  1TB SSD| 25.6| 8.0|20.3|   20.8|
+---+-----------+-----+-----------+----+---------+-----+----+----+-------+



In [18]:
sample_data_csv.printSchema()

root
 |-- Id: integer (nullable = true)
 |--  Model: string (nullable = true)
 |--  Year: integer (nullable = true)
 |--  ScreenSize: string (nullable = true)
 |--  RAM: string (nullable = true)
 |--  HDD: string (nullable = true)
 |--  W: double (nullable = true)
 |--  D: double (nullable = true)
 |--  H: double (nullable = true)
 |--  Weight: double (nullable = true)



### **Accessing underlying RDDs**

In [19]:
sample_data_df.rdd.take(1)

[Row(Id=1, Model='MacBook Pro', Year=2015, ScreenSize='15"', RAM='16GB', HDD='512GB SSD', W=13.75, D=9.48, H=0.61, Weight=4.02)]

In [20]:
sample_data.take(1)

[(1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD', 13.75, 9.48, 0.61, 4.02)]

In [21]:
import pyspark.sql as sql
import pyspark.sql.functions as f

sample_data_transformed = (
    sample_data_df
    .rdd
    .map(lambda row: sql.Row(
        **row.asDict(),
        HDD_size=row.HDD.split(' ')[0],
        HDD_type=row.HDD.split(' ')[1],
        Volume=row.H * row.D * row.W,
      )
    )
    .toDF()
    .select(
        sample_data_df.columns +
        [
            'HDD_size',
            'HDD_type',
            f.round(f.col('Volume')).alias('Volume_cuIn')
        ]
    )
)

sample_data_transformed.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+--------+--------+-----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|HDD_size|HDD_type|Volume_cuIn|
+---+-----------+----+----------+----+---------+-----+----+----+------+--------+--------+-----------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|   512GB|     SSD|       80.0|
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|   256GB|     SSD|       44.0|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|   128GB|     SSD|       78.0|
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|     1TB|     SSD|     4157.0|
+---+-----------+----+----------+----+---------+-----+----+----+------+--------+--------+-----------+



### **Performance optimizations**

In [22]:
import pyspark.sql.functions as f

big_df = (
    spark
    .range(0, 10000000)
    .withColumn('val', f.rand())
)

big_df.cache()
big_df.show(3)

+---+-------------------+
| id|                val|
+---+-------------------+
|  0|0.15824727011221507|
|  1| 0.7916534940264927|
|  2| 0.9268211161181736|
+---+-------------------+
only showing top 3 rows



In [23]:
import pandas as pd
from scipy import stats

@f.pandas_udf('double', f.PandasUDFType.SCALAR)
def pandas_pdf(v):
  return pd.Series(stats.norm.pdf(v))


def test_pandas_pdf():
  return (big_df
          .withColumn('probability', pandas_pdf(big_df.val))
          .agg(f.count(f.col('probability')))
          .show()
          )

%timeit -n 1 test_pandas_pdf()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+------------------+
|count(probability)|
+------------------+
|          10000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|          10000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|          10000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|          10000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|          10000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|          10000000|
+------------------+

+------------------+
|count(probability)|
+------------------+
|          10000000|
+------------------+

7.08 s ± 2.39 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
(
    big_df
    .withColumn('probability', pandas_pdf(big_df.val))
    .show(5)
)

+---+-------------------+-------------------+
| id|                val|        probability|
+---+-------------------+-------------------+
|  0|0.15824727011221507|  0.393978227070457|
|  1| 0.7916534940264927| 0.2916221969529596|
|  2| 0.9268211161181736|0.25964571187563923|
|  3| 0.7911086142069876| 0.2917479737666859|
|  4| 0.9076854937346297|0.26424329956613596|
+---+-------------------+-------------------+
only showing top 5 rows



In [26]:
@f.udf('double')
def pdf(v):
  return float(stats.norm.pdf(v))

def test_pdf():
  return (big_df
          .withColumn('probability', pdf(big_df.val))
          .agg(f.count(f.col('probability')))
        )

%timeit -n 1 test_pdf()

25.5 ms ± 6.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### **Inferring the schema using reflection**

In [37]:
sample_data_rdd = sc.textFile('DataFrames_sample.csv')
sample_data_rdd.take(2)

['Id, Model, Year, ScreenSize, RAM, HDD, W, D, H, Weight',
 '1,MacBook Pro,2015,"15\\"",16GB,512GB SSD,13.75,9.48,0.61,4.02']

In [38]:
import pyspark.sql as sql

header = sample_data_rdd.first()


sample_data_rdd_row = (
    sample_data_rdd
    .filter(lambda row: row != header)
    .map(lambda row: row.split(','))
    .map(lambda row:
         sql.Row(
             Id=int(row[0]),
             Model=row[1],
             Year=int(row[2]),
             ScreenSize=row[3],
             RAM=row[4],
             HDD=row[5],
             W=float(row[6]),
             D=float(row[7]),
             H=float(row[8]),
             Weight=float(row[9])
         )
    )
)

In [39]:
spark.createDataFrame(sample_data_rdd_row).show(3)

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
+---+-----------+----+----------+----+---------+-----+----+----+------+
only showing top 3 rows



### **Specifying the Schema programmatically**

**SparkSession : Point d'entrée pour les Dataframes**

In [50]:
import pyspark.sql.types as typ

sch = typ.StructType([

    typ.StructField('Id', typ.LongType(), False),
    typ.StructField('Model', typ.StringType(), True),
    typ.StructField('Year', typ.IntegerType(), True),
    typ.StructField('ScreenSize', typ.StringType(), True),
    typ.StructField('RAM', typ.StringType(), True),
    typ.StructField('HDD', typ.StringType(), True),
    typ.StructField('W', typ.DoubleType(), True),
    typ.StructField('D', typ.DoubleType(), True),
    typ.StructField('H', typ.DoubleType(), True),
    typ.StructField('Weight', typ.DoubleType(), True)
])

In [52]:
sample_data_rdd = sc.textFile('DataFrames_sample.csv')

header = sample_data_rdd.first()


sample_data_rdd = (
    sample_data_rdd
    .filter(lambda row: row != header)
    .map(lambda row: row.split(','))
    .map(lambda row:(
             int(row[0]),
             row[1],
             int(row[2]),
             row[3],
             row[4],
             row[5],
             float(row[6]),
             float(row[7]),
             float(row[8]),
             float(row[9])
         )
    )
)

sample_data_schema = spark.createDataFrame(sample_data_rdd, schema=sch)
sample_data_schema.show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



### **Creating a temporary table**

In [53]:
sample_data_schema.createTempView('sample_data_view')

In [55]:
spark.sql(
    '''
    SELECT Model,
          Year,
          RAM,
          HDD
    FROM sample_data_view
    '''
).show()


+-----------+----+----+---------+
|      Model|Year| RAM|      HDD|
+-----------+----+----+---------+
|MacBook Pro|2015|16GB|512GB SSD|
|    MacBook|2016| 8GB|256GB SSD|
|MacBook Air|2016| 8GB|128GB SSD|
|       iMac|2017|64GB|  1TB SSD|
+-----------+----+----+---------+



In [56]:
sample_data_schema.createOrReplaceTempView('sample_data_view')

In [58]:
spark.sql(
    '''
    SELECT Model,
          Year,
          RAM,
          HDD,
          ScreenSize
    FROM sample_data_view
    '''
).show()

+-----------+----+----+---------+----------+
|      Model|Year| RAM|      HDD|ScreenSize|
+-----------+----+----+---------+----------+
|MacBook Pro|2015|16GB|512GB SSD|    "15\""|
|    MacBook|2016| 8GB|256GB SSD|    "12\""|
|MacBook Air|2016| 8GB|128GB SSD|  "13.3\""|
|       iMac|2017|64GB|  1TB SSD|    "27\""|
+-----------+----+----+---------+----------+



### **Using SQL To Interact With DataFrames**

In [62]:
models_df = sc.parallelize([
    ('MacBook Pro', 'Laptop'),
    ('MacBook Air', 'Laptop'),
    ('MacBook', 'Laptop'),
    ('iMac', 'Desktop'),
]).toDF(['Model', 'FormFactor'])

models_df.createOrReplaceTempView('models')

In [63]:
sample_data_schema.createOrReplaceTempView('sample_data_view')

In [64]:
spark.sql(
    '''
    SELECT a.*,
            b.FormFactor
    FROM sample_data_view AS a
    LEFT JOIN models as b
      ON a.Model == b.Model
    ORDER BY Weight DESC
    '''
).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|FormFactor|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|   Desktop|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|    Laptop|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|    Laptop|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|    Laptop|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+



In [65]:
spark.sql(
    '''
    SELECT b.FormFactor,
          COUNT(*) AS ComputerCnt
    FROM sample_data_view AS a
    LEFT JOIN models as b
      ON a.Model == b.Model
      GROUP BY FormFactor
    '''
).show()

+----------+-----------+
|FormFactor|ComputerCnt|
+----------+-----------+
|    Laptop|          3|
|   Desktop|          1|
+----------+-----------+



### **OverView DataFrame transformations**

In [66]:
sample_data_schema.cache().show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [67]:
sample_data_schema.coalesce(1).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [68]:
sample_data_schema.select('RAM').distinct().show()

+----+
| RAM|
+----+
|16GB|
| 8GB|
|64GB|
+----+



In [69]:
sample_data_schema.dropDuplicates().show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [70]:
sample_data_schema.dropna().show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [75]:
models_df = sc.parallelize([
    ('MacBook Pro', 'Laptop'),
    ('MacBook Air', 'Laptop'),
    ('iMac', 'Desktop'),
]).toDF(['Model', 'FormFactor'])

(
    sample_data_schema
    .join(
        models_df,
        sample_data_schema.Model == models_df.Model,
        'left'
    ).show()
)

+---+-----------+----+----------+----+---------+-----+----+----+------+-----------+----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|      Model|FormFactor|
+---+-----------+----+----------+----+---------+-----+----+----+------+-----------+----------+
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|       NULL|      NULL|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|MacBook Pro|    Laptop|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|MacBook Air|    Laptop|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|       iMac|   Desktop|
+---+-----------+----+----------+----+---------+-----+----+----+------+-----------+----------+



In [76]:
(
    sample_data_schema
    .join(
        models_df,
        sample_data_schema.Model == models_df.Model,
        'right'
    ).show()
)

+---+-----------+----+----------+----+---------+-----+----+----+------+-----------+----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|      Model|FormFactor|
+---+-----------+----+----------+----+---------+-----+----+----+------+-----------+----------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|MacBook Pro|    Laptop|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|MacBook Air|    Laptop|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|       iMac|   Desktop|
+---+-----------+----+----------+----+---------+-----+----+----+------+-----------+----------+



In [78]:
(
    sample_data_schema
    .join(
        models_df,
        sample_data_schema.Model == models_df.Model,
        'left_anti'
    ).show()
)

+---+-------+----+----------+---+---------+-----+----+----+------+
| Id|  Model|Year|ScreenSize|RAM|      HDD|    W|   D|   H|Weight|
+---+-------+----+----------+---+---------+-----+----+----+------+
|  2|MacBook|2016|    "12\""|8GB|256GB SSD|11.04|7.74|0.52|  2.03|
+---+-------+----+----------+---+---------+-----+----+----+------+



In [79]:
(
    sample_data_schema
    .join(
        models_df,
        sample_data_schema.Model == models_df.Model,
        'left_semi'
    ).show()
)

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [80]:
sample_data_schema.fillna(0).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [70]:
sample_data_schema.fillna(0).show()

[(('City', 'IATA'), 0),
 (('Abbotsford', 'YXX'), 1),
 (('Aberdeen', 'ABR'), 2),
 (('Abilene', 'ABI'), 3),
 (('Akron', 'CAK'), 4)]

In [78]:
sample_data_schema.filter(sample_data_schema.Year > 2015).show()

[('Abbotsford', 'YXX'),
 ('Aberdeen', 'ABR'),
 ('Abilene', 'ABI'),
 ('Akron', 'CAK'),
 ('Alamosa', 'ALS')]

In [81]:
sample_data_schema.freqItems(['RAM']).show()

+-----------------+
|    RAM_freqItems|
+-----------------+
|[16GB, 64GB, 8GB]|
+-----------------+



In [82]:
sample_data_schema.orderBy('W').show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [83]:
sample_data_schema.orderBy(f.col('H').desc()).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [89]:
sample_data_schema_rep = (
    sample_data_schema
    .repartition(2, 'Year')
)

sample_data_schema_rep.rdd.getNumPartitions()

2

In [90]:
missing_df = sc.parallelize([
  (None, 36.3, 24.2),
  (1.6, 32.1, 27.9),
  (3.2, 38.7, 24.7),
  (2.8, None, 23.9),
  (3.9, 34.1, 27.9),
  (9.2, None, None)
]).toDF(['A', 'B', 'C'])

missing_df.fillna(21.4).show()

+----+----+----+
|   A|   B|   C|
+----+----+----+
|21.4|36.3|24.2|
| 1.6|32.1|27.9|
| 3.2|38.7|24.7|
| 2.8|21.4|23.9|
| 3.9|34.1|27.9|
| 9.2|21.4|21.4|
+----+----+----+



In [91]:
missing_df.dropna(thresh=2).show()

+----+----+----+
|   A|   B|   C|
+----+----+----+
|NULL|36.3|24.2|
| 1.6|32.1|27.9|
| 3.2|38.7|24.7|
| 2.8|NULL|23.9|
| 3.9|34.1|27.9|
+----+----+----+



In [92]:
missing_df.dropna().show()

+---+----+----+
|  A|   B|   C|
+---+----+----+
|1.6|32.1|27.9|
|3.2|38.7|24.7|
|3.9|34.1|27.9|
+---+----+----+



In [93]:
sample_data_schema.select('Model', 'ScreenSize').show()

+-----------+----------+
|      Model|ScreenSize|
+-----------+----------+
|MacBook Pro|    "15\""|
|    MacBook|    "12\""|
|MacBook Air|  "13.3\""|
|       iMac|    "27\""|
+-----------+----------+



In [94]:
sample_data_schema.select('W').summary().show()
sample_data_schema.select('W').describe().show()

+-------+------------------+
|summary|                 W|
+-------+------------------+
|  count|                 4|
|   mean|15.797500000000001|
| stddev| 6.630738395281983|
|    min|             11.04|
|    25%|             11.04|
|    50%|              12.8|
|    75%|             13.75|
|    max|              25.6|
+-------+------------------+

+-------+------------------+
|summary|                 W|
+-------+------------------+
|  count|                 4|
|   mean|15.797500000000001|
| stddev| 6.630738395281983|
|    min|             11.04|
|    max|              25.6|
+-------+------------------+



In [95]:
another_macbookPro = sc.parallelize([
    (5, 'MacBook Pro', 2018, '15"', '16GB', '256GB SSD', 13.75, 9.48, 0.61, 4.02)
]).toDF(sample_data_schema.columns)

sample_data_schema.unionAll(another_macbookPro).show()

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
|  5|MacBook Pro|2018|       15"|16GB|256GB SSD|13.75|9.48|0.61|  4.02|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [96]:
(
    sample_data_schema
    .withColumn('HDDSplit', f.split(f.col('HDD'), ' '))
    .show()
)

+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|    HDDSplit|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|[512GB, SSD]|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|[256GB, SSD]|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|[128GB, SSD]|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|  [1TB, SSD]|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+



In [97]:
(
    sample_data_schema
    .groupBy('RAM')
    .count()
    .show()
)

+----+-----+
| RAM|count|
+----+-----+
|16GB|    1|
| 8GB|    2|
|64GB|    1|
+----+-----+



In [99]:
(
    sample_data_schema
    .select(
        f.col('*'),
        f.split(f.col('HDD'), ' ').alias('HDD_Array'))
    .show()
)

+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|   HDD_Array|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
|  1|MacBook Pro|2015|    "15\""|16GB|512GB SSD|13.75|9.48|0.61|  4.02|[512GB, SSD]|
|  2|    MacBook|2016|    "12\""| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|[256GB, SSD]|
|  3|MacBook Air|2016|  "13.3\""| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|[128GB, SSD]|
|  4|       iMac|2017|    "27\""|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|  [1TB, SSD]|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+



### **Overview Of DataFrames actions**

In [100]:
sample_data_schema.groupBy('Year').count().collect()

[Row(Year=2015, count=1), Row(Year=2016, count=2), Row(Year=2017, count=1)]

In [101]:
sample_data_schema.select('W').describe().show()

+-------+------------------+
|summary|                 W|
+-------+------------------+
|  count|                 4|
|   mean|15.797500000000001|
| stddev| 6.630738395281983|
|    min|             11.04|
|    max|              25.6|
+-------+------------------+



In [102]:
sample_data_schema.take(2)

[Row(Id=1, Model='MacBook Pro', Year=2015, ScreenSize='"15\\""', RAM='16GB', HDD='512GB SSD', W=13.75, D=9.48, H=0.61, Weight=4.02),
 Row(Id=2, Model='MacBook', Year=2016, ScreenSize='"12\\""', RAM='8GB', HDD='256GB SSD', W=11.04, D=7.74, H=0.52, Weight=2.03)]

In [103]:
sample_data_schema.toPandas()

,Id,Model,Year,ScreenSize,RAM,HDD,W,D,H,Weight
0,1,MacBook Pro,2015,"""15\""""",16GB,512GB SSD,13.75,9.48,0.61,4.02
1,2,MacBook,2016,"""12\""""",8GB,256GB SSD,11.04,7.74,0.52,2.03
2,3,MacBook Air,2016,"""13.3\""""",8GB,128GB SSD,12.80,8.94,0.68,2.96
3,4,iMac,2017,"""27\""""",64GB,1TB SSD,25.60,8.00,20.30,20.80


In [105]:
sample_data_schema.write.mode('overwrite').csv('sample_data_schema.csv')